# Выбор модели

Можно разделить задачу кластеризации на 2 подзадачи:
1. Векторизация текстовых данных.
2. Кластеризация векторов данных.

## Векторизация текста

Для векторизации текста можно использовать множество подходов. От простых:
* **One-hot вектор**. Матрица с 1 напротив соответствующего слова, отальные 0. Но есть недостатки в хранении больших разряженных векторов, также не учитывает контекст и схожесть слов. Можно немного улучшить через дополнительную лемматизацию или стемминг, чтобы "Экономический" и "Экономические" давали одно слово "Экономика".<br />Это не самое оптимальное решение, используется, как мне кажется, только в RNN или трнасформерах, чтобы посчитать кросс-энтропию функции потерь. 
* **Bag of Words**. Метод аналогичен One-hot вектору, только вместо 1 для слова записывается количество его повторений в каждом тексте. Благодаря количеству, можно уже сложить какое-то базовое представлени о сути текста, но тоже не идеально. Недостатки такие же, но есть дополнительный - служебные частицы "и", "а", "но" и т.д. будут иметь большой вес, но по факту смысла для понимания сути текста не несут. Фиксится удалением таких слов на этапе токенизации.
* **TF-IDF**. Самый продвинутый из простых, даже без дополнительных этапов токенизации способен убирать "лишние" слова и оставлять только те, которые уникальны, отражают суть текста. Снова проблемы разряженности векторов, отсуствие контекста и не понимание схожести слов (снова можно частично пофиксить лемматизацией и стеммингом).

До более сложных, контекстных эмбеддингов:
* **Word2Vec/FastText** - варианты решений, как понятно из названия, понимающие контекст. Точнее, чем прошлые варианты, но всё равно из-за простоты архитектуры (пару Dense слоёв) не дают точного результата.
* **ELMo/Seq2Vec** - имеют более сложную архитекруту, используют рекуррентные слои, даже двунаправленные LSTM. Срединный вариант по ресурсам и качеству.
* **Transformers (LLM)** - SOTA (state-of-the-art) решение. Т.к. архитектура трансформеров построена на механизме *Attention*, они лучше всех понимают контекст, поэтому выдают лучшие эмбеддинги на данный момент. Можно охарактеризовать так: лучшее качество за ваши ресурсы.

В результате, я бы предложил следующие варианты:
* TF-IDF - когда нужно сделать супер-просто, супер-быстро с минимальными затратами ресурсов, но можно пожертвовать качеством.
* ELMo/Transformers - когда нужно использовать SOTA решение. Есть ресурсы, есть время, нужно качество.


Для решения текущей задачи с кластеризацией заголовков я бы выбрал какую-то небольшую LLM, которая понимает русский язык. Нашёл на Hugging Face одну для примера: https://huggingface.co/apanc/russian-sensitive-topics

## Кластеризация векторов

Вообще можно выбрать любой метод кластеризации, но я бы не хотел использовать K-means и т.п. с фиксированным количество кластеров, хочется посмотреть, как модель сама определит количество кластеров.

На собеседовании упоминался DBSCAN, если правильно помню. Возьму его.

# Загрузка данных и основных библиотек

In [416]:
# Куда без этого
import numpy as np
import pandas as pd

# Сам модуль кластеризации и методы для оценки качества
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Модули для загрузки Трансформеров (эмбеддинг и суммаризация)
import torch
from transformers import AutoModel, AutoTokenizer

In [417]:
text_data = pd.read_excel('./data/dataset.xlsx', header=None)

text_data

,0,1
0,NaN,Эксперты назвали сложности перехода Финляндии ...
1,NaN,Доходы duty free в аэропорту Владивостока выро...
2,NaN,Аналитики оценили вероятность первого с начала...
3,NaN,Какие стартапы могут потеснить Nvidia на рынке...
4,NaN,Партия Morena объявила о победе Шейнбаум на вы...
5,NaN,Республиканцы выдвинут Трампа кандидатом на вы...
6,NaN,«Зенит» выиграл Суперфинал Кубка России по фут...
7,NaN,Бывший губернатор Ханты-Мансийского автономног...
8,NaN,"""Реал"" стал победителем Лиги чемпионов, обыгра..."
9,NaN,"Силуанов выразил мнение, что россияне не будут..."


"Причешем" датасет

In [418]:
text_data.columns = ["Nulls", "text"]
text_data = text_data.drop('Nulls', axis=1)
text_data

,text
0,Эксперты назвали сложности перехода Финляндии ...
1,Доходы duty free в аэропорту Владивостока выро...
2,Аналитики оценили вероятность первого с начала...
3,Какие стартапы могут потеснить Nvidia на рынке...
4,Партия Morena объявила о победе Шейнбаум на вы...
5,Республиканцы выдвинут Трампа кандидатом на вы...
6,«Зенит» выиграл Суперфинал Кубка России по фут...
7,Бывший губернатор Ханты-Мансийского автономног...
8,"""Реал"" стал победителем Лиги чемпионов, обыгра..."
9,"Силуанов выразил мнение, что россияне не будут..."


# Эмбеддинг

Загрузим модель и её токенайзер

https://huggingface.co/apanc/russian-sensitive-topics

In [419]:
model_name = 'apanc/russian-sensitive-topics'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Основная функция эмбеддинга

In [420]:
def get_embeddings(texts: list[str]) -> np.ndarray:
    """Векторизация текстов с помощью эмбеддинга

    Args:
        texts (list[str]): Список текстов

    Returns:
        np.ndarray: Массив эмбеддингов размером NxK, где N - количество текстов в входном списке, K - размер скрытого слоя модели.
    """

    # Сначала токенизируем текст, т.е. разобьём его на части и почистим
    tokenized_texts = [tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True) for text in texts]

    max_tokens_num = max(len(tokens) for tokens in tokenized_texts) # опеределяем максимальную длину входных токенов для модели
    padded_token_ids = np.array([tokens + [0] * (max_tokens_num - len(tokens)) for tokens in tokenized_texts]) # приводим все векторы токенов к одной длине, добавляя пустые значения

    input_tokens = torch.tensor(padded_token_ids) # переводим токены из 2D массива в тензор

    with torch.inference_mode(): # без вычисления градиентов, т.к. у нас инференс
        outputs = model(input_tokens)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    
    return embeddings


Получение эмбеддингов. Так как эмбеддинги получаются не нормализованные, а методы, которые используют расстояния, чувствительны к разным шкалам измерения, лучше выполнить нормализацию.

In [421]:
text_embeddings = get_embeddings(text_data['text'].values)

text_embeddings

array([[ 0.01307951,  0.06388409,  1.6944324 , ..., -0.1813996 ,
         0.8761033 , -1.5702151 ],
       [-0.4839972 ,  0.04623525,  1.8314607 , ..., -0.1510437 ,
         1.4248095 , -1.2298682 ],
       [-0.13476014,  0.04057334,  1.4830717 , ..., -0.34455973,
         1.5135832 , -1.1993951 ],
       ...,
       [ 0.29948887, -0.76610965,  1.3187362 , ...,  0.10979811,
         0.3620852 , -1.5645237 ],
       [-0.04394834, -0.61255187,  0.5913162 , ..., -0.21642065,
        -0.48199937, -1.1076522 ],
       [ 0.26521274,  0.18564923,  1.6109005 , ...,  0.19758587,
         1.5585958 , -0.8093844 ]], dtype=float32)

# Кластеризация

Инициализация модели

In [422]:
# Параметры DBSCAN
eps = .2  # Радиус окрестности
min_samples = 2  # Минимальное количество точек в кластере
metric = 'cosine' # По своему опыту могу сделать вывод, что среди расстояний Левенштейна, Евклида и косинусного, лучше всего себя показавало именно косинусное 

In [423]:
cluster_model = DBSCAN(eps = eps, min_samples = min_samples, metric = metric)

In [424]:
# Кластеризация
cluster_labels = cluster_model.fit_predict(text_embeddings)
cluster_labels

array([0, 0, 0, 0, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0])

In [425]:
text_data["cluster_label"] = cluster_labels

text_data

,text,cluster_label
0,Эксперты назвали сложности перехода Финляндии ...,0
1,Доходы duty free в аэропорту Владивостока выро...,0
2,Аналитики оценили вероятность первого с начала...,0
3,Какие стартапы могут потеснить Nvidia на рынке...,0
4,Партия Morena объявила о победе Шейнбаум на вы...,1
5,Республиканцы выдвинут Трампа кандидатом на вы...,1
6,«Зенит» выиграл Суперфинал Кубка России по фут...,2
7,Бывший губернатор Ханты-Мансийского автономног...,1
8,"""Реал"" стал победителем Лиги чемпионов, обыгра...",2
9,"Силуанов выразил мнение, что россияне не будут...",1


# Оценка качества кластеризации

In [426]:
metrics = {
    'Silhouette': silhouette_score(text_embeddings, cluster_labels),
    'Davies Bouldin Index': davies_bouldin_score(text_embeddings, cluster_labels)
}

metrics

{'Silhouette': 0.49763802, 'Davies Bouldin Index': 0.636097358228092}

В результате кластеризации мы получили 3 класса. Судя по статьям, можно выделить следующую закономерность:
* 0 - экономические статьи
* 1 - политические статьи
* 2 - спортивные статьи

Судя по метрикам качества кластеризации, модель хорошо разделила классы:
* Силуэт - **0.5**. Принимает значения $[-1;1]$, где ближе к -1 - всё плохо, 0 - объекты находятся на границе, 1 - объекты хорошо разделены. Можно сделать вывод, что наше разделение получилось хорошим.
* Индекс Дэвиса-Булдина - **0.64**. Принимает значения $[0; -\infty]$. Обычно находится в диапазоне $[0; 3]$. Чем ближе к 0, тем лучше кластеризация. Наша кластеризация достаточно хорошая. 

# Сохранение результатов

In [427]:
text_data.to_csv('./data/results.csv', index=False)